# Content-based recommendations (using cosine similarity)
This notebook shows the process used to calculate recommendations for user based on their previous ratings and in features extracted from content texts available in +TV4E platform.

Library requirements:
* Pandas
* NumPy
* Scikit-learn 
* NLTK (w/ portuguese stop words)

In [1]:
# for Python 2: use print only as a function
from __future__ import print_function

### Part 1 - Examine the data

In [2]:
# read CSV using a relative path
import pandas as pd
import numpy as np
import requests
URL_RATINGS='http://api_mysql.tv4e.pt/api/recommendations/ratings'
data=requests.get(URL_RATINGS)
df_ratings=pd.DataFrame(data.json())
URL_VIDEOS='http://api_mysql.tv4e.pt/api/recommendations/videos'
data= requests.get(URL_VIDEOS)
df_videos=pd.DataFrame(data.json())

In [3]:
# show dataframe first rows
df_videos.head()

,video_asgie_id,video_asgie_title_en,video_asgie_title_pt,video_date_creation,video_desc,video_id,video_location,video_title
0,3,Financial Services,Finanças,2017-09-26 15:21:16,Seja qual for o cenário para os escalões do IR...,4049,,As “habilidadezinhas” do Governo para descer o...
1,1,Health Care and Welfare Services,Saúde e Bem-Estar,2017-09-26 15:36:20,Mais pessoas recorrem aos centros de saúde par...,4051,,Deixar de fumar
2,1,Health Care and Welfare Services,Saúde e Bem-Estar,2017-09-26 15:38:36,Quase sete mil unidades de medicamentos apreen...,4053,,Medicamentos falsificados
3,1,Health Care and Welfare Services,Saúde e Bem-Estar,2017-09-26 15:40:12,Ministro da Saúde participa no encontro em Coi...,4054,,Conferência Mundial de Saúde Urbana
4,1,Health Care and Welfare Services,Saúde e Bem-Estar,2017-09-26 15:40:19,Rastreio visa despiste da ambliopia em criança...,4055,,CHTS | Saúde visual infantil


In [4]:
# show dataframe first rows
df_ratings.head()

,rating_date_creation,rating_value,user_id,video_id,video_watch_time,video_watched_type
0,2017-10-23 00:00:00,0,2,4355,1,notified
1,2017-10-17 10:05:45,0,3,4275,1,notified
2,2017-09-27 14:00:04,0,5,4090,1,notified
3,2017-09-27 13:30:03,0,5,4091,1,notified
4,2017-09-27 13:00:03,0,5,4092,1,notified


In [5]:
df_ratings[['user_id', 'video_id', 'rating_value']]

,user_id,video_id,rating_value
0,2,4355,0
1,3,4275,0
2,5,4090,0
3,5,4091,0
4,5,4092,0
5,5,4093,0
6,5,4094,0
7,5,4095,0
8,5,4107,0
9,5,4108,0


In [6]:
#show the shape of dataframe
df_ratings.shape

(42, 6)

In [7]:
# show overall statistics of the dataframe
df_ratings.describe()

,rating_value,user_id,video_id,video_watch_time
count,42.0,42.000000,42.000000,42.000000
mean,0.0,6.428571,4113.119048,4.142857
std,0.0,1.939862,56.480172,10.045194
min,0.0,2.000000,4051.000000,1.000000
25%,0.0,5.000000,4090.250000,1.000000
50%,0.0,5.000000,4094.500000,1.000000
75%,0.0,8.000000,4113.750000,1.000000
max,0.0,9.000000,4355.000000,49.000000


In [8]:
# filter ratings from user 1 and show statistics
df_ratings_user = df_ratings[(df_ratings.user_id==1)]
df_ratings_user.describe()

,rating_value,user_id,video_id,video_watch_time
count,0.0,0.0,0.0,0.0
mean,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,NaN
max,NaN,NaN,NaN,NaN


### Part 2 - Clean ratings data

In [9]:
# rtemoving unused columns
df_ratings.drop('video_watched_type', axis=1, inplace=True)
df_ratings.drop('rating_date_creation', axis=1, inplace=True)
df_ratings

,rating_value,user_id,video_id,video_watch_time
0,0,2,4355,1
1,0,3,4275,1
2,0,5,4090,1
3,0,5,4091,1
4,0,5,4092,1
5,0,5,4093,1
6,0,5,4094,1
7,0,5,4095,1
8,0,5,4107,1
9,0,5,4108,1


In [10]:
# calculate implicit and explicit ratings
# XXX use a function to calculate implicit rating considering the video lead time
df_ratings['rating_implicit'] = (df_ratings['video_watch_time']/100)*0.3
df_ratings['rating_explicit'] = (df_ratings['rating_value'])        *0.7

# create a new column to put implicit or explicit rating value
df_ratings['overall_rating_value'] = df_ratings['rating_implicit'] + df_ratings['rating_explicit']
# OLD: if there's no explicit rating, consider the implicit rating as 
# df_ratings['overall_rating_value'] = np.where(df_ratings['rating_value'] == 0, df_ratings['video_watch_time'], df_ratings['rating_value'])
df_ratings


,rating_value,user_id,video_id,video_watch_time,rating_implicit,rating_explicit,overall_rating_value
0,0,2,4355,1,0.003,0.0,0.003
1,0,3,4275,1,0.003,0.0,0.003
2,0,5,4090,1,0.003,0.0,0.003
3,0,5,4091,1,0.003,0.0,0.003
4,0,5,4092,1,0.003,0.0,0.003
5,0,5,4093,1,0.003,0.0,0.003
6,0,5,4094,1,0.003,0.0,0.003
7,0,5,4095,1,0.003,0.0,0.003
8,0,5,4107,1,0.003,0.0,0.003
9,0,5,4108,1,0.003,0.0,0.003


In [11]:
ratings_matrix = df_ratings.pivot_table(index=['user_id'], columns=['video_id'], values=['overall_rating_value'])
ratings_matrix                                     

overall_rating_value                                            \
video_id                 4051   4061   4062   4063   4068   4089   4090   
user_id                                                                   
2                         NaN    NaN    NaN    NaN    NaN    NaN    NaN   
3                         NaN    NaN    NaN    NaN    NaN    NaN    NaN   
5                         NaN    NaN    NaN    NaN    NaN    NaN  0.003   
8                       0.039  0.003  0.003  0.003  0.003  0.006  0.003   
9                         NaN    NaN    NaN    NaN  0.012  0.024  0.003   

                               ...                                       \
video_id   4091   4092   4093  ...     4115   4116   4117   4118   4172   
user_id                        ...                                        
2           NaN    NaN    NaN  ...      NaN    NaN    NaN    NaN    NaN   
3           NaN    NaN    NaN  ...      NaN    NaN    NaN    NaN    NaN   
5         0.003  0.003  0.003  ...    0.003  0.003  0.003  0.003  0.003   
8         0.003    NaN  0.147  ...      NaN    NaN    NaN    NaN    NaN   
9         0.003    NaN  0.003  ...      NaN    NaN    NaN    NaN    NaN   

                                             
video_id   4173   4174   4220   4275   4355  
user_id                                      
2           NaN    NaN    NaN    NaN  0.003  
3           NaN    NaN    NaN  0.003    NaN  
5         0.027  0.135    NaN    NaN    NaN  
8           NaN    NaN    NaN    NaN    NaN  
9           NaN    NaN  0.003    NaN    NaN  

[5 rows x 30 columns]

### Part 3 - Vectorize texts

In [12]:
# import and retrieve portuguese stop words
# stop words are not considered as token as usually they carry no meaning (!)
from nltk.corpus import stopwords
portuguese_stop_words = stopwords.words('portuguese')

In [13]:
# import and instantiate TF-IDF Vectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1, 2), min_df=2, max_df=0.5, stop_words=portuguese_stop_words)

In [14]:
# fit (train) the vectorizer with the corpus from video text contents
tfidf_matrix = tfidf.fit_transform(df_videos['video_desc'])
tfidf_matrix.toarray()

array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [15]:
tfidf.get_feature_names()

['00',
 '000',
 '000 00',
 '000 euros',
 '018',
 '018 2017',
 '06',
 '09',
 '10',
 '10 10saúdealarga',
 '10 aniversário',
 '10 anos',
 '10 horas',
 '10 meses',
 '10 mil',
 '10 milhões',
 '10 novembro',
 '10 outubro',
 '100',
 '100 aquisição',
 '100 mil',
 '102',
 '10h30',
 '10h30 fundação',
 '10saúdealarga',
 '10saúdealarga elenco',
 '11',
 '11 novembro',
 '11 outubro',
 '110',
 '112',
 '112 acordo',
 '112 instituto',
 '112 recorda',
 '113',
 '113 2011',
 '118',
 '12',
 '12 anos',
 '12 meses',
 '12 mil',
 '12 outubro',
 '120',
 '120 médicos',
 '125',
 '127',
 '13',
 '13 milhões',
 '13 novembro',
 '13 outubro',
 '130',
 '131',
 '131 2017',
 '14',
 '14 17',
 '14 mil',
 '14 novembro',
 '140',
 '140 2016',
 '142',
 '14h',
 '14h30',
 '15',
 '15 16',
 '15 horas',
 '15 milhões',
 '15 novembro',
 '15 outubro',
 '15 pontos',
 '150',
 '150 000',
 '150 200',
 '150 mil',
 '150 milhões',
 '154',
 '154 administração',
 '154 profissionais',
 '154 vagas',
 '157',
 '15h00',
 '16',
 '16 horas',
 '16 mil

### Part 4 - Build up user profile

In [16]:
# create dict video_id ==>> tfidf weights
tfidf_tokens = {}
tfidf_array = tfidf_matrix.toarray()
line_count = 0
for idx, row in df_videos.iterrows():
    tfidf_tokens[row.video_id] = tfidf_array[line_count]
    line_count += 1
tfidf_tokens

{4049: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 4051: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 4053: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 4054: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 4055: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 4057: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 4058: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 4059: array([ 0.        ,  0.18206838,  0.        , ...,  0.        ,
         0.        ,  0.        ]),
 4060: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 4061: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 4062: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 4063: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 4068: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 4070: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 4071: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 4072: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 4073: array([ 0.,  0.,  0., ...,  0.,  0.,  0.]),
 4074: array([ 0.        ,  0.        ,  0.        , ...,  0.        ,
      

In [17]:
# define user ratings
user_ratings = df_ratings[(df_ratings.user_id==8)]
user_ratings

,rating_value,user_id,video_id,video_watch_time,rating_implicit,rating_explicit,overall_rating_value
23,0,8,4051,13,0.039,0.0,0.039
24,0,8,4061,1,0.003,0.0,0.003
25,0,8,4062,1,0.003,0.0,0.003
26,0,8,4063,1,0.003,0.0,0.003
27,0,8,4068,1,0.003,0.0,0.003
28,0,8,4089,2,0.006,0.0,0.006
29,0,8,4090,1,0.003,0.0,0.003
30,0,8,4091,1,0.003,0.0,0.003
31,0,8,4093,49,0.147,0.0,0.147
32,0,8,4094,7,0.021,0.0,0.021


In [18]:
# created weighted user profile vector (dotproduct of vectors of items consumed and user ratings)
#   "In the original implementation, the profile was the sum of the item-tag vectors of all items 
#   the user has rated positively (>= 3.5 stars). This approach was later improved with weighted 
#   user profile (with the older implementation commented out for reference). Weighted profile is 
#   computed with weighted sum of the item vectors for all items, with weights being based on the 
#   user's rating."
#   See: http://eugenelin89.github.io/recommender_content_based/
user_profile = [0] * len(tfidf.get_feature_names())
for i in range(len(user_profile)):
    for idx, row in user_ratings.iterrows():
        user_profile[i] += row.overall_rating_value * tfidf_tokens[row.video_id][i]
    #user_profile = [v/len(user_ratings) for v in user_profile] # weight-ing user vector (?)
# normalize user profile vector
import numpy as np
user_profile = user_profile / np.linalg.norm(user_profile)
user_profile

array([ 0.,  0.,  0., ...,  0.,  0.,  0.])

### Part 5 - Calculate similarities between user profile and contents

In [19]:
# apply cosine similarity between user profile vector and content vectors
# See: http://eugenelin89.github.io/recommender_content_based/
from math import sqrt
def my_cosine_similarity(x, y):
    def square_rooted(v):
        return round(sqrt(sum([a * a for a in v])), 3)
   
    numerator = sum(a * b for a, b in zip(x, y))
    denominator = square_rooted(x) * square_rooted(y)
    return numerator/float(denominator)

estimated_user_ratings = {}
for video_id, token_weights in tfidf_tokens.items(): 
    if video_id not in user_ratings.video_id.values: # not calculating for contents already consumed 
        estimated_user_ratings[video_id] = my_cosine_similarity(user_profile, token_weights)
# order ratings
import operator
estimated_user_ratings = sorted(estimated_user_ratings.items(), key=operator.itemgetter(1))
estimated_user_ratings[:-10:-1]

[(4556, 0.17264066507438913),
 (4450, 0.16207649580126726),
 (4454, 0.14924849521371653),
 (4097, 0.14527711276733171),
 (4515, 0.12871522418028294),
 (4227, 0.12677987355095829),
 (4195, 0.12484667926298997),
 (4157, 0.12094561789942655),
 (4148, 0.12001665899329583)]

In [20]:
df_videos[(df_videos.video_id == 4148)].video_date_creation.values

array(['2017-10-03 10:31:30'], dtype=object)

In [24]:
# Using sklearn cosine_similarity (faster!)
from sklearn.metrics.pairwise import cosine_similarity
from math import sqrt
estimated_user_ratings = []
for video_id, token_weights in tfidf_tokens.items(): 
    # not calculating for contents already consumed
    if video_id not in user_ratings.video_id.values:
        estimated_user_ratings.append((
            video_id,
            df_videos[(df_videos.video_id == video_id)].video_date_creation.values[0],
            cosine_similarity([user_profile], [token_weights])[0]
        ))
#     if video_id not in user_ratings.video_id.values: # not calculating for contents already consumed 
#         estimated_user_ratings[video_id] = cosine_similarity([user_profile], [token_weights])
# order ratings
estimated_user_ratings = sorted(estimated_user_ratings, key=lambda tup: tup[2], reverse=True)[:10]
estimated_user_ratings

[(4556, '2017-11-20 10:28:14', array([ 0.17264067])),
 (4450, '2017-11-14 11:28:05', array([ 0.1620765])),
 (4454, '2017-11-14 11:28:34', array([ 0.1492485])),
 (4097, '2017-09-27 14:23:24', array([ 0.14527711])),
 (4515, '2017-11-16 13:12:49', array([ 0.12871522])),
 (4227, '2017-10-12 11:30:56', array([ 0.12677987])),
 (4195, '2017-10-09 00:35:27', array([ 0.12484668])),
 (4157, '2017-10-04 00:27:48', array([ 0.12094562])),
 (4148, '2017-10-03 10:31:30', array([ 0.12001666])),
 (4432, '2017-11-06 01:13:33', array([ 0.11813912]))]

In [22]:
# Using the dotproduct of user profile vector, content IDF and content vector
# See: https://www.analyticsvidhya.com/blog/2015/08/beginners-guide-learn-content-based-recommender-systems/
estimated_user_ratings = {}
for video_id, token_weights in tfidf_tokens.items():
    if video_id not in user_ratings.video_id.values: # removing contents already consumed 
        estimated_user_ratings[video_id] = 0
        for i in range(len(token_weights)):
            if token_weights[i] > 0:
                estimated_user_ratings[video_id] += user_profile[i] * tfidf.idf_[i] * token_weights[i]
# order ratings
import operator
estimated_user_ratings = sorted(estimated_user_ratings.items(), key=operator.itemgetter(1))
estimated_user_ratings[:-10:-1]

[(4450, 0.73881498788373268),
 (4556, 0.69866128136157435),
 (4097, 0.63346060494261824),
 (4148, 0.48295080099898696),
 (4421, 0.4815366917222561),
 (4454, 0.4668566657643804),
 (4515, 0.46588342560886353),
 (4157, 0.44962354975368696),
 (4195, 0.43920300622765879)]